In [1]:
# Import dependencies
import pandas as pd
from collections import Counter

In [2]:
# Open the file
tourney_csv = "MNCAATourneyDetailedResults.csv"
tourney_df = pd.read_csv(tourney_csv)

tourney_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


In [3]:
# Check the number of unique winning team IDs
tourney_df['WTeamID'].value_counts().count()

163

In [4]:
# Create a df with only unique winning TeamIDs
w_team_ids_df = tourney_df['WTeamID'].drop_duplicates()
len(w_team_ids_df)

163

In [5]:
# Create a df with only unique losing TeamIDs
l_team_ids_df = tourney_df['LTeamID'].drop_duplicates()
len(l_team_ids_df)

254

In [6]:
# Combine the winning and losing team ID dfs into one and drop duplicate team IDs
team_ids = pd.concat([w_team_ids_df, l_team_ids_df]).drop_duplicates()
len(team_ids)

254

In [7]:
# Convert to df with TeamID as the index
team_ids_df = pd.DataFrame(team_ids).rename(columns={0:'TeamID'}).set_index('TeamID')
team_ids_df.head()

""
TeamID
1421
1112
1113
1141
1143


In [8]:
# Import MTeams.csv to bring in team info
teams_csv = "MTeams.csv"
teams_df = pd.read_csv(teams_csv)

teams_df.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2022
1,1102,Air Force,1985,2022
2,1103,Akron,1985,2022
3,1104,Alabama,1985,2022
4,1105,Alabama A&M,2000,2022


In [9]:
# Set index to Team ID
teams_df.set_index('TeamID',inplace=True)
teams_df.head()

,TeamName,FirstD1Season,LastD1Season
TeamID,,,
1101,Abilene Chr,2014,2022
1102,Air Force,1985,2022
1103,Akron,1985,2022
1104,Alabama,1985,2022
1105,Alabama A&M,2000,2022


In [10]:
# Get the number of tournament wins
num_wins = tourney_df['WTeamID'].value_counts()
num_wins_df = pd.DataFrame(num_wins).rename(columns={'WTeamID':'NumTournW'})

num_wins_df.head()

,NumTournW
1314,45
1242,44
1246,40
1181,39
1277,36


In [11]:
# Join team ID with num_wins_df to get the number of wins in the tournament
team_stats_df = teams_df.join(num_wins_df, how='left')
team_stats_df.head()

,TeamName,FirstD1Season,LastD1Season,NumTournW
TeamID,,,,
1101,Abilene Chr,2014,2022,1.0
1102,Air Force,1985,2022,NaN
1103,Akron,1985,2022,NaN
1104,Alabama,1985,2022,7.0
1105,Alabama A&M,2000,2022,NaN


In [12]:
# Replace NaN with 0
team_stats_df['NumTournW'] = team_stats_df['NumTournW'].fillna(0)
team_stats_df.head()

,TeamName,FirstD1Season,LastD1Season,NumTournW
TeamID,,,,
1101,Abilene Chr,2014,2022,1.0
1102,Air Force,1985,2022,0.0
1103,Akron,1985,2022,0.0
1104,Alabama,1985,2022,7.0
1105,Alabama A&M,2000,2022,0.0


In [13]:
# Get the number of tournament losses
num_losses = tourney_df['LTeamID'].value_counts()
num_losses_df = pd.DataFrame(num_losses).rename(columns={'LTeamID':'NumTournL'})

num_losses_df.head()

,NumTournL
1211,18
1277,18
1458,17
1242,17
1181,15


In [14]:
# Join df with num_losses_df to add the number of tournament losses
team_stats_df = team_stats_df.join(num_losses_df, how='left')
team_stats_df.head()

,TeamName,FirstD1Season,LastD1Season,NumTournW,NumTournL
TeamID,,,,,
1101,Abilene Chr,2014,2022,1.0,2.0
1102,Air Force,1985,2022,0.0,2.0
1103,Akron,1985,2022,0.0,3.0
1104,Alabama,1985,2022,7.0,7.0
1105,Alabama A&M,2000,2022,0.0,1.0


In [15]:
# View stats
team_stats_df.describe()

,FirstD1Season,LastD1Season,NumTournW,NumTournL
count,372.000000,372.000000,372.000000,254.000000
mean,1988.887097,2021.053763,3.174731,4.649606
std,8.469902,5.216324,7.247947,3.885625
min,1985.000000,1985.000000,0.000000,1.000000
25%,1985.000000,2022.000000,0.000000,2.000000
50%,1985.000000,2022.000000,0.000000,3.000000
75%,1985.000000,2022.000000,2.000000,7.000000
max,2022.000000,2022.000000,45.000000,18.000000


In [16]:
# Add a column for number of tournament games played
team_stats_df['NumTournGames'] = team_stats_df['NumTournW'] + team_stats_df['NumTournL']

team_stats_df.head()

,TeamName,FirstD1Season,LastD1Season,NumTournW,NumTournL,NumTournGames
TeamID,,,,,,
1101,Abilene Chr,2014,2022,1.0,2.0,3.0
1102,Air Force,1985,2022,0.0,2.0,2.0
1103,Akron,1985,2022,0.0,3.0,3.0
1104,Alabama,1985,2022,7.0,7.0,14.0
1105,Alabama A&M,2000,2022,0.0,1.0,1.0


In [17]:
# Add a column for winning percentages
team_stats_df['TournWinPct'] = round(team_stats_df['NumTournW']/team_stats_df['NumTournGames'],2)

team_stats_df.head()

,TeamName,FirstD1Season,LastD1Season,NumTournW,NumTournL,NumTournGames,TournWinPct
TeamID,,,,,,,
1101,Abilene Chr,2014,2022,1.0,2.0,3.0,0.33
1102,Air Force,1985,2022,0.0,2.0,2.0,0.00
1103,Akron,1985,2022,0.0,3.0,3.0,0.00
1104,Alabama,1985,2022,7.0,7.0,14.0,0.50
1105,Alabama A&M,2000,2022,0.0,1.0,1.0,0.00


In [18]:
# Average FGA in winning game
test2 = tourney_df.groupby(['WTeamID'])['WFGA'].mean()
test2.head()

WTeamID
1101    67.000000
1104    53.142857
1107    47.000000
1112    56.227273
1113    49.000000
Name: WFGA, dtype: float64

In [19]:
# Create a df of max winning scores
winning_points_df = tourney_df[['WTeamID','WScore']].set_index('WTeamID')
winning_points_df = winning_points_df.max(level=0)
winning_points_df.head()

C:\Users\z002cr1\AppData\Local\Temp\1\ipykernel_23904\3729402199.py:3: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  winning_points_df = winning_points_df.max(level=0)


,WScore
WTeamID,
1421,92
1112,100
1113,84
1141,79
1143,77


In [20]:
# Create a df of the max losing score per team
losing_points_df = tourney_df[['LTeamID','LScore']].set_index('LTeamID')
losing_points_df = losing_points_df.max(level=0)                                                             
losing_points_df.head()

C:\Users\z002cr1\AppData\Local\Temp\1\ipykernel_23904\3949310770.py:3: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  losing_points_df = losing_points_df.max(level=0)


,LScore
LTeamID,
1411,84
1436,70
1272,91
1166,73
1301,83


In [21]:
# Merge the winning and losing max scores dfs
max_scores_df = losing_points_df.join(winning_points_df, how='left').fillna(0)
max_scores_df.head()

,LScore,WScore
LTeamID,,
1411,84,64.0
1436,70,71.0
1272,91,94.0
1166,73,76.0
1301,83,79.0


In [22]:
# Bring max score into stats df
team_stats_df['MaxTournScore'] = max_scores_df[['WScore','LScore']].max(axis=1)

team_stats_df.head()

,TeamName,FirstD1Season,LastD1Season,NumTournW,NumTournL,NumTournGames,TournWinPct,MaxTournScore
TeamID,,,,,,,,
1101,Abilene Chr,2014,2022,1.0,2.0,3.0,0.33,53.0
1102,Air Force,1985,2022,0.0,2.0,2.0,0.00,69.0
1103,Akron,1985,2022,0.0,3.0,3.0,0.00,64.0
1104,Alabama,1985,2022,7.0,7.0,14.0,0.50,96.0
1105,Alabama A&M,2000,2022,0.0,1.0,1.0,0.00,69.0


In [23]:
# Create a df of min winning scores
winning_points_min_df = tourney_df[['WTeamID','WScore']].set_index('WTeamID')
winning_points_min_df = winning_points_min_df.min(level=0)
winning_points_min_df.head()

C:\Users\z002cr1\AppData\Local\Temp\1\ipykernel_23904\2695006255.py:3: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  winning_points_min_df = winning_points_min_df.min(level=0)


,WScore
WTeamID,
1421,81
1112,66
1113,66
1141,79
1143,64


In [24]:
# Create a df of the min losing score per team
losing_points_min_df = tourney_df[['LTeamID','LScore']].set_index('LTeamID')
losing_points_min_df = losing_points_min_df.min(level=0)                                                             
losing_points_min_df.head()

C:\Users\z002cr1\AppData\Local\Temp\1\ipykernel_23904\2361816778.py:3: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  losing_points_min_df = losing_points_min_df.min(level=0)


,LScore
LTeamID,
1411,64
1436,51
1272,45
1166,50
1301,54


In [25]:
# Merge the winning and losing min scores dfs
min_scores_df = losing_points_min_df.join(winning_points_min_df, how='left')
min_scores_df.head()

,LScore,WScore
LTeamID,,
1411,64,60.0
1436,51,60.0
1272,45,54.0
1166,50,58.0
1301,54,58.0


In [26]:
# Bring min score into stats df
team_stats_df['MinTournScore'] = min_scores_df[['WScore','LScore']].min(axis=1)

team_stats_df.head(20)

,TeamName,FirstD1Season,LastD1Season,NumTournW,NumTournL,NumTournGames,TournWinPct,MaxTournScore,MinTournScore
TeamID,,,,,,,,,
1101,Abilene Chr,2014,2022,1.0,2.0,3.0,0.33,53.0,44.0
1102,Air Force,1985,2022,0.0,2.0,2.0,0.00,69.0,52.0
1103,Akron,1985,2022,0.0,3.0,3.0,0.00,64.0,42.0
1104,Alabama,1985,2022,7.0,7.0,14.0,0.50,96.0,57.0
1105,Alabama A&M,2000,2022,0.0,1.0,1.0,0.00,69.0,69.0
1106,Alabama St,1985,2022,0.0,3.0,3.0,0.00,61.0,43.0
1107,SUNY Albany,2000,2022,1.0,5.0,6.0,0.17,71.0,55.0
1108,Alcorn St,1985,2022,0.0,NaN,NaN,NaN,NaN,NaN
1109,Alliant Intl,1985,1991,0.0,NaN,NaN,NaN,NaN,NaN


In [28]:
team_stats_df.columns

Index(['TeamName', 'FirstD1Season', 'LastD1Season', 'NumTournW', 'NumTournL',
       'NumTournGames', 'TournWinPct', 'MaxTournScore', 'MinTournScore'],
      dtype='object')

In [29]:
# Refine df to schools with last season in D1 2003+
filtered_team_stats_df = team_stats_df.loc[team_stats_df['LastD1Season'] >= 2003]
filtered_team_stats_df.head(20)

,TeamName,FirstD1Season,LastD1Season,NumTournW,NumTournL,NumTournGames,TournWinPct,MaxTournScore,MinTournScore
TeamID,,,,,,,,,
1101,Abilene Chr,2014,2022,1.0,2.0,3.0,0.33,53.0,44.0
1102,Air Force,1985,2022,0.0,2.0,2.0,0.00,69.0,52.0
1103,Akron,1985,2022,0.0,3.0,3.0,0.00,64.0,42.0
1104,Alabama,1985,2022,7.0,7.0,14.0,0.50,96.0,57.0
1105,Alabama A&M,2000,2022,0.0,1.0,1.0,0.00,69.0,69.0
1106,Alabama St,1985,2022,0.0,3.0,3.0,0.00,61.0,43.0
1107,SUNY Albany,2000,2022,1.0,5.0,6.0,0.17,71.0,55.0
1108,Alcorn St,1985,2022,0.0,NaN,NaN,NaN,NaN,NaN
1110,American Univ,1985,2022,0.0,3.0,3.0,0.00,67.0,35.0


In [30]:
# Replace NaN with 0
filtered_team_stats_df = filtered_team_stats_df.fillna(0)

filtered_team_stats_df.head(20)

,TeamName,FirstD1Season,LastD1Season,NumTournW,NumTournL,NumTournGames,TournWinPct,MaxTournScore,MinTournScore
TeamID,,,,,,,,,
1101,Abilene Chr,2014,2022,1.0,2.0,3.0,0.33,53.0,44.0
1102,Air Force,1985,2022,0.0,2.0,2.0,0.00,69.0,52.0
1103,Akron,1985,2022,0.0,3.0,3.0,0.00,64.0,42.0
1104,Alabama,1985,2022,7.0,7.0,14.0,0.50,96.0,57.0
1105,Alabama A&M,2000,2022,0.0,1.0,1.0,0.00,69.0,69.0
1106,Alabama St,1985,2022,0.0,3.0,3.0,0.00,61.0,43.0
1107,SUNY Albany,2000,2022,1.0,5.0,6.0,0.17,71.0,55.0
1108,Alcorn St,1985,2022,0.0,0.0,0.0,0.00,0.0,0.0
1110,American Univ,1985,2022,0.0,3.0,3.0,0.00,67.0,35.0
